In [194]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.robust import scale, norms
import pyreadstat
import statsmodels.api as sm
from patsy import dmatrices
import nbformat
from nbconvert import PythonExporter

from typing import List, Optional, Dict, Any


os.chdir('/Users/connorbrennan/OneDrive - The University of Chicago/mmb/data')

In [195]:
df, meta = pyreadstat.read_dta('derived/MMB_reg_format.dta')
df_labels = pd.DataFrame({
    "Variable": meta.column_names,
    "Description": meta.column_labels
})
df = df.loc[df['y_timing_max'] < 99]
df = df.loc[df['piq_timing_max'] < 99]

print(df_labels)
df['not_pr_ndx'] = 1 - df['pr_ndx']
df['not_wg_ndx'] = 1 - df['wg_ndx']
#df = df.loc[df['sacratio20'] < df['sacratio20'].quantile(0.98)]
df_estimated = df.loc[df['estimated']==1]
df_calibrated = df.loc[df['calibrated']==1]

           Variable           Description
0             model         (first) model
1              rule          (first) rule
2           rule_tr           Taylor Rule
3          rule_itr  Inertial Taylor Rule
4            rule_g           Growth Rule
..              ...                   ...
129   stky_wg_nondx                  None
130        stky_all                  None
131  stky_pr_wg_ndx                  None
132         ndx_all    P and W Indexation
133          ln_neq       ln(Num. of Eq.)

[134 rows x 2 columns]


In [196]:
fixed_vars = ['rule_g', 'rule_itr', 'estimated']
depvars_elasticities = ['IScurve20', 'infl_per_rr20', 'sacratio20']
depvars_timing = ['y_timing_max', 'piq_timing_max']

In [197]:
def generate_latex_tables(stepwise_regs, r2_values, depvars, horizons, var_labels, depvar_labels, outfile=None):
    """
    Given a dictionary of stepwise regression results (stepwise_regs), a list of
    dependent variables (depvars), and a list of horizons, generate nicely
    formatted LaTeX tables with multirow rows for each variable, variable labels,
    and lines at the bottom for R^2 or nobs.

    We double backslash LaTeX commands to avoid Python interpreting escape chars.
    """
    latex_pieces = []
    for depvar in depvars:
        # 2A) Identify which models belong to this dependent variable
        these_models = {}
        for h in horizons:
            key = f"{depvar}{h}"
            if key in stepwise_regs:
                these_models[h] = stepwise_regs[key]

        # If no models found for this depvar, skip
        if not these_models:
            continue

        # 2B) Collect the union of all variable names across these models
        varset = set()
        for h, model in these_models.items():
            varset = varset.union(model.params.index)
        # Sort them in a consistent order, but keep 'Intercept' on top if you prefer
        varlist = sorted(varset, key=lambda v: (v != 'Intercept', v))

        # 2C) Start building the LaTeX string
        latex_str = []
        latex_str.append("\\begin{table}[h!]")
        latex_str.append("\\centering")
        latex_str.append("\\resizebox{0.8\\textwidth}{!}{%")  # Optional scaling
        latex_str.append("\\begin{tabular}{l" + "c"*len(horizons) + "}")
        latex_str.append("\\hline")

        # 2D) First row: label the dependent variable, spanning all columns
        latex_str.append(
            f"\\multicolumn{{{len(horizons)+1}}}{{l}}{{\\textbf{{Dependent Variable: {depvar_labels[depvar]}}}}} \\\\"
        )
        latex_str.append("\\hline")

        # 2E) Print horizon labels, e.g. & (20) & (40) & (60)
        horizon_header = "\\textbf{{Horizon}} & " + " & ".join([f"{h}" for h in horizons]) + " \\\\"
        latex_str.append(horizon_header)
        latex_str.append("\\hline")

        # 2F) For each variable, produce two rows:
        #     (1) multirow w/ var name + coefficients
        #     (2) blank first cell + std errors
        for var in varlist:
            # Build dict for param/std_err across horizons
            coeffs = []
            std_errs = []
            for h in horizons:
                model = these_models.get(h, None)
                if model is not None and var in model.params.index:
                    param = model.params[var]
                    pval  = model.pvalues[var]
                    stderr= model.bse[var]

                    coeffs.append(format_coef(param, pval))
                    std_errs.append(format_se(stderr))
                else:
                    coeffs.append("")
                    std_errs.append("")

            # 2G) Resolve variable label if available, otherwise default
            if var in var_labels:
                varname_latex = var_labels[var]
            else:
                varname_latex = var
            # Escape underscores for LaTeX
            varname_latex = varname_latex.replace("_", "\\_")

            # Multirow lines
            line1 = f"\\multirow{{2}}{{*}}{{{varname_latex}}} & " + " & ".join(coeffs) + " \\\\"
            line2 = " & " + " & ".join(std_errs) + " \\\\"

            latex_str.append(line1)
            latex_str.append(line2)

        # 2H) Now we add lines for number of observations, and (optionally) R^2
        #     We'll build them across the horizons, e.g.: Observations & n1 & n2 & n3
        #     For RLM, there's no built-in R^2, but we can just show placeholders
        latex_str.append("\\hline")

        # Observations line
        nobs_list = []
        for h in horizons:
            model = these_models.get(h, None)
            if model is not None:
                nobs_list.append(str(int(model.nobs)))
            else:
                nobs_list.append("")
        latex_str.append("Observations & " + " & ".join(nobs_list) + " \\\\")

        # R-squared (placeholder or a custom statistic)
        # For RLM there's no rsquared by default, so we just show an example row:
        # If you compute your own pseudo-R^2, replace "... " with that value
        r2_list = []
        for h in horizons:
            r2 = r2_values.get(f'{depvar}{h}', None)
            if r2 is not None:
                # placeholder; replace with something like "f'{model_custom_r2:.3f}'"
                r2_list.append(str(round(r2,3)))
            else:
                r2_list.append("")
        latex_str.append("$R^2$ from Final Weights& " + f'{" & ".join(r2_list)}' + " \\\\")

        # 2I) Wrap up
        latex_str.append("\\hline")
        latex_str.append("\\end{tabular}")
        latex_str.append("}")  # Closes \\resizebox
        latex_str.append(f"\\caption{{Stepwise RLM results for {depvar_labels[depvar]}}}")
        latex_str.append("\\end{table}")
        latex_str.append("\\newpage")

        # Print the LaTeX code for this table
        table_code = "\n".join(latex_str)
        latex_pieces.append(table_code)
        #print(table_code)
        #print("\n\n")  # some space after each table
    
    all_tables = "\n\n".join(latex_pieces)
    if outfile is not None:
        with open(outfile, "w") as f:
            f.write(all_tables)
        print(f"Saved all tables to {outfile}")
    else:
        print(all_tables)
        print("\n\n")



def generate_latex_master_table(stepwise_regs, r2_values, depvars,
                                var_labels, depvar_labels, outfile=None,
                                table_caption="Regression results (all models)"):
    """
    Build ONE LaTeX table with columns for each depvar model.
    Rows are the union of variables across all models. Blank cells where
    a variable didn't enter. Two rows per variable: coef row, then std. error row.

    Requires helper functions:
        - format_coef(beta, pval) -> str
        - format_se(stderr) -> str
    """

    # 1) Determine which depvars exist in results
    columns = []
    for depvar in depvars:
        if depvar in stepwise_regs:
            columns.append(depvar)

    if not columns:
        print("No models found in stepwise_regs for the provided depvars.")
        return

    # 2) Union of all variable names across all models
    varset = set()
    for depvar in columns:
        varset |= set(stepwise_regs[depvar].params.index)

    # Keep Intercept/const on top, then alphabetical
    varlist = sorted(varset, key=lambda v: (v not in ("Intercept","const"), v))

    # 3) Build LaTeX
    latex = []
    latex.append("\\begin{table}[h!]")
    latex.append("\\centering")
    latex.append("\\resizebox{0.95\\textwidth}{!}{%")
    latex.append("\\setlength{\\tabcolsep}{8pt}%")
    latex.append("\\begin{tabular}{l" + "c"*len(columns) + "}")
    latex.append("\\hline")

    # 4) Header row with depvar names
    header_cells = ["\\textbf{Variable}"]
    for depvar in columns:
        label = depvar_labels.get(depvar, depvar).replace("_", "\\_")
        header_cells.append(f"\\textbf{{{label}}}")
    latex.append(" & ".join(header_cells) + " \\\\")
    latex.append("\\hline")

    # 5) Body: each variable, coef row + se row
    for var in varlist:
        coef_cells = [var_labels.get(var, var).replace("_", "\\_")]
        for depvar in columns:
            model = stepwise_regs[depvar]
            if var in model.params.index:
                beta = model.params[var]
                pval = model.pvalues.get(var, None)
                coef_cells.append(format_coef(beta, pval))
            else:
                coef_cells.append("")
        latex.append(" & ".join(coef_cells) + " \\\\")

        se_cells = [""]
        for depvar in columns:
            model = stepwise_regs[depvar]
            if var in model.params.index:
                se = model.bse.get(var, None)
                se_cells.append(format_se(se))
            else:
                se_cells.append("")
        latex.append(" & ".join(se_cells) + " \\\\")

    latex.append("\\hline")

    # 6) Observations row
    obs_cells = ["Observations"]
    for depvar in columns:
        model = stepwise_regs[depvar]
        obs_cells.append(str(int(getattr(model, "nobs", "")) or ""))
    latex.append(" & ".join(obs_cells) + " \\\\")

    # 7) R^2 row
    r2_cells = ["$R^2$"]
    for depvar in columns:
        r2 = r2_values.get(depvar, None)
        r2_cells.append("" if r2 is None else f"{r2:.3f}")
    latex.append(" & ".join(r2_cells) + " \\\\")

    latex.append("\\hline")
    latex.append("\\end{tabular}")
    latex.append("}% end resizebox")
    latex.append(f"\\caption{{{table_caption}}}")
    latex.append("\\end{table}")
    latex.append("\\newpage")

    table_code = "\n".join(latex)

    if outfile is not None:
        with open(outfile, "w") as f:
            f.write(table_code)
        print(f"Saved master table to {outfile}")
    else:
        print(table_code)



def format_coef(param, pval):
    """
    Formats the coefficient with 3 decimal places plus significance stars.
    """
    return f"{param:.3f}{significance_stars(pval)}"



def format_se(std_err):
    """
    Formats the standard error in parentheses, with 3 decimal places.
    """
    return f"({std_err:.3f})"



def significance_stars(pval):
    if pval < 0.01:
        return "***"
    elif pval < 0.05:
        return "**"
    elif pval < 0.10:
        return "*"
    else:
        return ""
    

def get_r2(orig_reg, depvar, data):
    """
    Unified R^2:
      - RLM: adj. R^2 via WLS with final weights.
      - GLM (Poisson / NegBin): deviance-based pseudo-R^2 if available,
        else McFadden's pseudo-R^2.
      - Fallback: OLS adj. R^2 or NaN.
    """
    import numpy as np
    import pandas as pd
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    from patsy import dmatrices

    # ---- GLM path (Poisson / NegBin etc.) ----
    # GLMResults have .model.family, .deviance, .null_deviance, .llf, .llnull
    if hasattr(orig_reg, "model") and hasattr(orig_reg.model, "family"):
        res = orig_reg
        r2_dev = None
        r2_mcf = None

        null_dev = getattr(res, "null_deviance", None)
        dev = getattr(res, "deviance", None)
        if null_dev is not None and dev is not None and np.isfinite(null_dev) and null_dev > 0:
            r2_dev = 1.0 - (dev / null_dev)

        llnull = getattr(res, "llnull", None)
        llf = getattr(res, "llf", None)
        if llnull is not None and llf is not None and np.isfinite(llnull) and llnull != 0:
            r2_mcf = 1.0 - (llf / llnull)

        # Prefer deviance-based; fallback to McFadden
        if r2_dev is not None and np.isfinite(r2_dev):
            return float(r2_dev)
        if r2_mcf is not None and np.isfinite(r2_mcf):
            return float(r2_mcf)
        return np.nan

    # ---- RLM path (your robust linear models) ----
    # Rebuild WLS with final weights to compute adj. R^2 (what you had before).
    if hasattr(orig_reg, "weights") and hasattr(orig_reg, "params"):
        terms = [t for t in orig_reg.params.index if t not in ("Intercept", "const")]
        formula_str = f"{depvar} ~ " + (" + ".join(terms) if terms else "1")

        y, X = dmatrices(formula_str, data=data, return_type="dataframe")
        valid_index = X.index

        # orig_reg.weights may be array-like; make a Series aligned to valid_index
        if isinstance(orig_reg.weights, pd.Series):
            weights_series = orig_reg.weights.loc[valid_index]
        else:
            weights_series = pd.Series(np.asarray(orig_reg.weights), index=valid_index)

        df_sub = data.loc[valid_index]
        wls_reg = smf.wls(formula_str, data=df_sub, weights=weights_series).fit(cov="H1")
        return float(wls_reg.rsquared_adj)

    # ---- Fallback: OLS adj. R^2 using available params as terms ----
    try:
        terms = [t for t in orig_reg.params.index if t not in ("Intercept", "const")]
        formula_str = f"{depvar} ~ " + (" + ".join(terms) if terms else "1")
        ols_reg = smf.ols(formula_str, data=data).fit()
        return float(ols_reg.rsquared_adj)
    except Exception:
        return float("nan")

In [198]:
depvar_labels = {'IScurve20': 'IS Curve',
                 'infl_per_rr20': 'Pi Curve',
                 'sacratio20': 'Sacrifice Ratio',
                 'y_timing_max': 'y-timing ',
                 'piq_timing_max': 'pi_timing'}



var_labels = {
    'Intercept': "Constant",
    'rule_g': "Rule: Growth",
    'rule_itr': "Rule: Inert. Taylor",
    'stky_pr_calvo': "Sticky Prices (Calvo)",
    'stky_pr_rotemberg': "Sticky Prices (Rotemberg)", 
    'stky_pr_other': "Sticky Prices (Other)", 
    'stky_wg': "Sticky Wages", 
    'pr_ndx': "Price Idx", 
    'wg_ndx': "Wage Idx.", 
    'wg_ndx_prprice': "Wage Idx. (Prev. Price)", 
    'wg_ndx_mult': "Wage Idx. (Mult. Price)", 
    'wg_ndx_other': "Wage Idx. (Other)",

    # interactions (nomrig)
    'stky_pr_calvo:pr_ndx': "Sticky Price (Calvo) $\\times$ Price Idx.", 
    'stky_pr_rotemberg:pr_ndx': "Sticky Price (Rotemberg) $\\times$ Price Idx.", 
    'stky_pr_other:pr_ndx': "Sticky Price (Other) $\\times$ Price Idx.",
    'stky_wg:wg_ndx': "Sticky Wages $\\times$ Wage Idx.", 
    'stky_wg:wg_ndx_prprice': "Sticky Wages $\\times$ Wage Idx. (Prev. Price)", 
    'stky_wg:wg_ndx_mult': "Sticky Wages $\\times$ Wage Idx. (Mult. Price)", 
    'stky_wg:wg_ndx_other': "Sticky Wages $\\times$ Wage Idx. (Other)",

    # added missing nomrig
    'stky_pr': "Sticky Prices",
    'stky_wg': "Sticky Wages",
    'stky_pr:pr_ndx': "Sticky Prices $\\times$ Price Idx.",
    'stky_wg:wg_ndx': "Sticky Wages $\\times$ Wage Idx.",
    'stky_pr:not_pr_ndx': "Sticky Prices $\\times$ Not Price Idx.",
    'stky_wg:not_wg_ndx': "Sticky Wages $\\times$ Not Wage Idx.",
    'stky_pr:wg_ndx': "Sticky Prices $\\times$ Wage Idx.",
    'stky_wg:pr_ndx': "Sticky Wages $\\times$ Price Idx.",
    'stky_pr:not_wg_ndx': "Sticky Prices $\\times$ Not Wage Idx.",
    'stky_wg:not_pr_ndx': "Sticky Wages $\\times$ Not Price Idx.",

    # real rigidities
    'wlth': "Wealth Channel",
    'ntwrth': "Net Worth Channel",
    'bnkcrdit': "Bank Credit Channel",
    'open': "Open Economy",
    'learning': "Learning Channel",

    'wlth:ntwrth': "Wealth $\\times$ Net Worth",
    'wlth:bnkcrdit': "Wealth $\\times$ Bank Credit",
    'wlth:open': "Wealth $\\times$ Open Economy",
    'wlth:learning': "Wealth $\\times$ Learning",
    'ntwrth:bnkcrdit': "Net Worth $\\times$ Bank Credit",
    'ntwrth:open': "Net Worth $\\times$ Open Economy",
    'ntwrth:learning': "Net Worth $\\times$ Learning",
    'bnkcrdit:open': "Bank Credit $\\times$ Open Economy",
    'bnkcrdit:learning': "Bank Credit $\\times$ Learning",
    'open:learning': "Open Economy $\\times$ Learning",

    # non-modeling vars
    'estimated': "Estimated", 
    'est_early': "Early Data", 
    'est_late': "Late Data", 
    'vint_early': "Early Vintage", 
    'vint_mid': "Mid Vintage", 
    'vint_late': "Late Vintage", 
    'cb_authors_ext': "Central Bank Author", 
    'ln_neq': "$\\log(\\text{Num. of Eqs.})$",

    # interactions (nonmod)
    'cb_authors_ext:estimated': "Central Bank Author $\\times$ Estimated",
    'cb_authors_ext:ln_neq': "Central Bank Author $\\times$ $\\log($Num. of Eqs.$)$",
    'cb_authors_ext:vint_early': "Central Bank Author $\\times$ Early Vintage",
    'cb_authors_ext:vint_mid': "Central Bank Author $\\times$ Mid Vintage",
    'cb_authors_ext:vint_late': "Central Bank Author $\\times$ Late Vintage",
    'cb_authors_ext:est_early': "Central Bank Author $\\times$ Early Data",
    'cb_authors_ext:est_late': "Central Bank Author $\\times$ Late Data",

    'estimated:ln_neq': "Estimated $\\times$ $\\log($Num. of Eqs.$)$",
    'estimated:vint_early': "Estimated $\\times$ Early Vintage",
    'estimated:vint_mid': "Estimated $\\times$ Mid Vintage",
    'estimated:vint_late': "Estimated $\\times$ Late Vintage",
    'estimated:est_early': "Estimated $\\times$ Early Data",
    'estimated:est_late': "Estimated $\\times$ Late Data",

    'ln_neq:vint_early': "$\\log($Num. of Eqs.$)$ $\\times$ Early Vintage",
    'ln_neq:vint_mid': "$\\log($Num. of Eqs.$)$ $\\times$ Mid Vintage",
    'ln_neq:vint_late': "$\\log($Num. of Eqs.$)$ $\\times$ Late Vintage",
    'ln_neq:est_early': "$\\log($Num. of Eqs.$)$ $\\times$ Early Data",
    'ln_neq:est_late': "$\\log($Num. of Eqs.$)$ $\\times$ Late Data",

    'vint_early:est_early': "Early Vintage $\\times$ Early Data",
    'vint_early:est_late': "Early Vintage $\\times$ Late Data",
    'vint_mid:est_early': "Mid Vintage $\\times$ Early Data",
    'vint_mid:est_late': "Mid Vintage $\\times$ Late Data",
    'vint_late:est_early': "Late Vintage $\\times$ Early Data",
    'vint_late:est_late': "Late Vintage $\\times$ Late Data"
}

In [199]:
def build_formula(vars_, depvar, fixed_vars = fixed_vars):
    rhs = []
    if vars_:
        rhs.append("+".join(vars_))
    if fixed_vars:
        rhs.append("+".join(fixed_vars))
    return f"{depvar} ~ " + (" + ".join(rhs) if rhs else "1")

In [200]:
def wooldridge_overdispersion_glm(
    df: pd.DataFrame,
    dep: str,
    regressors: List[str],
    controls: Optional[List[str]] = fixed_vars,
    subset: Optional[pd.Series] = None,
    time_limit_col: Optional[str] = None,
    time_limit_val: Optional[float] = None,
    alpha_floor: float = 1e-8,
    robust_cov: str = "HC0"
):
    """
    Wooldridge (1997) overdispersion test + choose Poisson vs NB2 (QMLE with robust SEs).
    Uses Patsy formulas so interaction terms like 'a:b' are constructed properly.
    Returns the *final* fitted results object (Poisson or NB2) for your later use.
    """
    # Build mask analogous to Stata's if-conditions
    mask = pd.Series(True, index=df.index)
    if subset is not None:
        mask &= subset.astype(bool)
    if (time_limit_col is not None) and (time_limit_val is not None):
        mask &= df[time_limit_col] < time_limit_val

    # Build formula string (this already appends your fixed_vars)
    formula = build_formula(regressors, dep, fixed_vars=controls)

    # Use Patsy to build y, X with interactions handled
    data_sub = df.loc[mask].copy()
    y, X = dmatrices(formula, data=data_sub, return_type='dataframe')

    # --- Step 1: Robust Poisson GLM
    poisson_model = sm.GLM(y, X, family=sm.families.Poisson())
    pois_res = poisson_model.fit(cov_type=robust_cov)

    # μ-hat
    mu_hat = pois_res.fittedvalues.squeeze()

    # --- Step 2: Wooldridge auxiliary regression (no constant)
    aux_dep = ((y.squeeze() - mu_hat) ** 2) - y.squeeze()
    mu2 = mu_hat ** 2
    aux_model = sm.OLS(aux_dep, mu2)  # nocons
    aux_res = aux_model.fit(cov_type=robust_cov)
    alpha_hat = float(aux_res.params.squeeze())
    p_over = float(aux_res.pvalues.squeeze())

    # --- Step 3: Choose final model
    if np.isnan(alpha_hat):
        return pois_res

    if p_over < 0.05:
        alpha_use = max(alpha_hat, alpha_floor)
        nb_fam = sm.families.NegativeBinomial(alpha=alpha_use)  # Var = μ + α μ²
        nb_model = sm.GLM(y, X, family=nb_fam)
        final_res = nb_model.fit(cov_type=robust_cov)
        return final_res
    else:
        return pois_res

# Bill's Way -- only have signficantly estimated variables enter in

In [201]:
indepvars = ['stky_wg:wg_ndx', 'stky_wg:not_wg_ndx', 'pr_ndx', 
                          'ntwrth:bnkcrdit', 'wlth:ntwrth', 'wlth:open', 'ntwrth', 'wlth',
                          'cb_authors_ext', 'vint_early', 'vint_mid']

In [202]:
regs = {}
r2s = {}

for dvar in depvars_elasticities:
    y, X = dmatrices(build_formula(indepvars, dvar), data=df, return_type='dataframe')
    rank_X, cond_num = np.linalg.matrix_rank(X), np.linalg.cond(X)
    if (rank_X < X.shape[1]) or (cond_num > 1000):
        raise('Collinearity detected!')

    regs[f'{dvar}'] = smf.rlm(
                build_formula(indepvars, dvar),
                M = norms.TukeyBiweight(c=4.685),
                data = df
            ).fit(
                scale_est = scale.HuberScale(),
                update_scale = True,
                cov = 'H1',
                conv='coefs'
            )
    r2s[f'{dvar}'] = get_r2(regs[f'{dvar}'], f'{dvar}', df)
    


for dvar in depvars_timing:
    regs[f'{dvar}'] = wooldridge_overdispersion_glm(df, f'{dvar}', indepvars)
    r2s[f'{dvar}'] = get_r2(regs[f'{dvar}'], f'{dvar}', df)

generate_latex_master_table(regs, r2s, depvars_elasticities + depvars_timing, var_labels, depvar_labels, '../output/table11/billway.txt')


Saved master table to ../output/table11/billway.txt


# Connor's way 
If we do want to keep the RHSs the same, I believe what we have for Table 11 is practically fine. However, I believe we should also add net worth effect and open as their own regressors as is standard when doing interactions. This is going to lead into my final point below…

When including bivariate interaction terms, does it make sense to include only the multiplication and not the individual variables themselves? That is, not just including, say “Bank Credit *Net Worth” but also including “Net Worth” and “Bank Credit” on their own. Unless we have a good theoretical justification for why we should not include those individual terms, I believe we should include those in all regressions involving interactions in Table 11. Therefore, my specification would be to simply include bank credit, open, wealth, and net worth as their own regressors in Table 11. These suggestions are based off my took of Tables 7-10 and what stepwise suggests should work best.

My justification for Table 11 would be that, after examining Tables 7-10, we include variables that appear important across our three categories to see how much variation we can explain within the responses of these macro models. And it is not much, which is our main selling point for this section. Then we can say that “in general, models of a middle vintage are characterized as… while models with central bank authors are characterized as…”

In [203]:
indepvars = ['stky_wg:wg_ndx', 'stky_wg:not_wg_ndx', 'pr_ndx',
            'ntwrth:bnkcrdit', 'wlth:open', 'ntwrth', 'wlth', 'bnkcrdit', 'open',
            'cb_authors_ext', 'vint_mid', 'vint_late', 'cb_authors_ext:vint_late',]

In [204]:
regs = {}
r2s = {}

for dvar in depvars_elasticities:
    y, X = dmatrices(build_formula(indepvars, dvar), data=df, return_type='dataframe')
    rank_X, cond_num = np.linalg.matrix_rank(X), np.linalg.cond(X)
    if (rank_X < X.shape[1]) or (cond_num > 1000):
        raise('Collinearity detected!')

    regs[f'{dvar}'] = smf.rlm(
                build_formula(indepvars, dvar),
                M = norms.TukeyBiweight(c=4.685),
                data = df
            ).fit(
                scale_est = scale.HuberScale(),
                update_scale = True,
                cov = 'H1',
                conv='coefs'
            )
    r2s[f'{dvar}'] = get_r2(regs[f'{dvar}'], f'{dvar}', df)
    


for dvar in depvars_timing:
    regs[f'{dvar}'] = wooldridge_overdispersion_glm(df, f'{dvar}', indepvars)
    r2s[f'{dvar}'] = get_r2(regs[f'{dvar}'], f'{dvar}', df)

generate_latex_master_table(regs, r2s, depvars_elasticities + depvars_timing, var_labels, depvar_labels, '../output/table11/connorway.txt')


Saved master table to ../output/table11/connorway.txt


In [205]:
df['open'].mean()

0.11059907834101383